### Phase 1.1, 1.2, 1.3

In [1]:
from unsloth import FastLanguageModel
import torch

lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 4096,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-15 16:38:22 [__init__.py:244] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4. vLLM: 0.9.1.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.418 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-unsloth-bnb-4bit with actual GPU utilization = 69.27%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 44.42 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 27.91 GB. Also swap space 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-15 16:38:45 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 06-15 16:38:45 [gpu_model_runner.py:1624] Model loading took 3.5740 GiB and 2.429191 seconds
INFO 06-15 16:38:59 [backends.py:462] Using cache directory: /root/.cache/vllm/torch_compile_cache/9bced0cd06/rank_0_0 for vLLM's torch.compile
INFO 06-15 16:38:59 [backends.py:472] Dynamo bytecode transform time: 12.73 s
INFO 06-15 16:39:09 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 9.009 s
INFO 06-15 16:39:16 [monitor.py:34] torch.compile takes 12.73 s in total
INFO 06-15 16:39:17 [gpu_worker.py:227] Available KV cache memory: 25.37 GiB
INFO 06-15 16:39:17 [kv_cache_utils.py:715] GPU KV cache size: 184,704 tokens
INFO 06-15 16:39:17 [kv_cache_utils.py:719] Maximum concurrency for 4,096 tokens per request: 45.09x
INFO 06-15 16:40:38 [gpu_model_runner.py:2048] Graph capturing finished in 81 secs, took 1.48 GiB
INFO 06-15 16:40:39 [core.py:171] init engine (profile, create kv c

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


<a name="Data"></a>
### Data Prep
We will use a custom dataset named `Interview_Data_6K.csv`. This dataset contains conversations with a mental health counselling assistant. Each entry has an `instruction` (acting as a system prompt), an `input` (the user's message), and an `output` (the assistant's response).

We need to convert this CSV data into a format suitable for training with `SFTTrainer`, specifically by applying the Qwen3 chat template.

In [3]:
from datasets import load_dataset, DatasetDict

# Load the custom CSV dataset
# The dataset has 'instruction', 'input', 'output' columns
full_interview_dataset = load_dataset("csv", data_files="./dataset/Interview_Data_6K.csv", split="train")

# Split the dataset into training and evaluation sets (e.g., 90% train, 10% eval)
# Ensure the dataset has more than one example for splitting
if len(full_interview_dataset) > 1:
    train_test_split = full_interview_dataset.train_test_split(test_size=0.05, seed=3407)
    interview_train_dataset = train_test_split['train']
    interview_eval_dataset = train_test_split['test']
    print(f"Training set size: {len(interview_train_dataset)}")
    print(f"Evaluation set size: {len(interview_eval_dataset)}")
else:
    # Handle cases with very small datasets
    interview_train_dataset = full_interview_dataset
    interview_eval_dataset = None
    print(f"Training set size: {len(interview_train_dataset)}")
    print("No evaluation set created due to small dataset size.")

# We will display the structure of the training dataset in the next cell.

Training set size: 5994
Evaluation set size: 316


Let's see the structure of our loaded training dataset:

In [4]:
interview_train_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5994
})

In [5]:
import pandas as pd # For pd.notna

# 'interview_train_dataset' and 'interview_eval_dataset' are from the previous cell
# The 'tokenizer' is globally defined in a prior cell (Cell 6, id="c1180838")

def convert_csv_to_chat_format(examples):
    """
    Converts a batch of examples from the CSV structure to a list of conversations.
    Each conversation is a list of dictionaries with "role" and "content".
    """
    all_conversations = []
    # 'examples' is a dictionary where keys are column names and values are lists of entries
    num_examples = len(examples['instruction'])
    
    for i in range(num_examples):
        instruction = examples['instruction'][i]
        input_text = examples['input'][i]
        output_text = examples['output'][i]

        # Ensure input_text is a string; handle None or NaN by treating as empty string if necessary
        input_text_str = str(input_text) if pd.notna(input_text) and str(input_text).strip() else ""

        current_conversation = []
        # System prompt from 'instruction'
        current_conversation.append({"role": "system", "content": instruction})
        
        # User message from 'input'
        # Based on the dataset, 'input' should always be present.
        # If input_text_str is empty, this will add a user message with empty content.
        # This is generally fine as the model should learn to handle it or it implies
        # the system prompt itself is the query.
        current_conversation.append({"role": "user", "content": input_text_str})
            
        # Assistant message from 'output'
        current_conversation.append({"role": "assistant", "content": output_text})
        
        all_conversations.append(current_conversation)
        
    return {"conversations": all_conversations}

def apply_template_to_conversations(examples):
    """
    Applies the tokenizer's chat template to a batch of conversations.
    Creates a 'text' field for SFTTrainer.
    """
    # tokenizer should be globally available
    return {
        "text": tokenizer.apply_chat_template(
            examples["conversations"], # This is a list of conversations
            tokenize=False,
            add_generation_prompt=False, # Crucial for training
        )
    }

# Process training data
# 1. Convert CSV structure to list of message dicts
train_dataset_with_conversations = interview_train_dataset.map(
    convert_csv_to_chat_format,
    batched=True,
    remove_columns=interview_train_dataset.column_names # Keep only 'conversations'
)
# 2. Apply chat template to create the 'text' field
final_train_dataset = train_dataset_with_conversations.map(
    apply_template_to_conversations,
    batched=True,
    remove_columns=["conversations"] # Keep only 'text'
)
# 3. Shuffle the training dataset
final_train_dataset = final_train_dataset.shuffle(seed=3407)

# Process evaluation data (if it exists)
final_eval_dataset = None
if interview_eval_dataset:
    eval_dataset_with_conversations = interview_eval_dataset.map(
        convert_csv_to_chat_format,
        batched=True,
        remove_columns=interview_eval_dataset.column_names
    )
    final_eval_dataset = eval_dataset_with_conversations.map(
        apply_template_to_conversations,
        batched=True,
        remove_columns=["conversations"]
    )
    # No need to shuffle eval_dataset

print("Sample of final formatted training data (after chat template):")
if len(final_train_dataset) > 0:
    print(final_train_dataset[0]['text'])
else:
    print("Training dataset is empty after processing.")

if final_eval_dataset and len(final_eval_dataset) > 0:
    print("\nSample of final formatted evaluation data (after chat template):")
    print(final_eval_dataset[0]['text'])
elif interview_eval_dataset: # If interview_eval_dataset existed but final_eval_dataset is empty
    print("\nEvaluation dataset is empty after processing.")

Map:   0%|          | 0/5994 [00:00<?, ? examples/s]

Map:   0%|          | 0/5994 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Sample of final formatted training data (after chat template):
<|im_start|>system
You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. <|im_end|>
<|im_start|>user
I've always had a preference for fewer, but closer relationships with my family, as opposed to more casual friendships outside of it. However, I often feel starved for company, isolated, and unhappy being withdrawn from social activities. Yet, I find it difficult to make friends and have only a few real ones in my life.<|im_end|>
<|im_start|>assistant
<think>

</think>

Your preference for closer relationships within your family is understandable, as it's natural to crave intimacy and connection. However, your feelings of loneliness and isolation when it comes to your social life are valid as well. It's not uncommon for individuals to struggle with findin

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [6]:
import wandb
wandb.init(project="distress-chatbot", name="base-model-traininf-v1_1", config={
    "model": "Qwen/Qwen3-4B",
    # "max_steps": 20000,
    "learning_rate": 2e-5,
    "lambda_decay": 0.95,
})  # Allow resuming W&B run

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_dataset, # Use the processed training data
    eval_dataset = final_eval_dataset,   # Use the processed evaluation data
    args = SFTConfig(
        dataset_text_field = "text", # Column containing the formatted text
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        # max_steps = 60, # Adjusted for quicker testing, was 30. Set to None or higher for full training.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
        # evaluation_strategy = "steps" if final_eval_dataset else "no", # Enable evaluation if eval_dataset is present
        # eval_steps = 20, # Evaluate every N steps, adjust as needed
    ),
)

# If using evaluation, you might want to set evaluation_strategy and eval_steps in SFTConfig
if final_eval_dataset:
    trainer.args.evaluation_strategy = "steps"
    trainer.args.eval_steps = 20 # Or any other desired frequency
else:
    trainer.args.evaluation_strategy = "no"

wandb: Currently logged in as: hongtai91 (hongtai91-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Tokenizing ["text"] (num_proc=256):   0%|          | 0/5994 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=256):   0%|          | 0/316 [00:00<?, ? examples/s]

In [7]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L40S. Max memory = 44.418 GB.
31.408 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,994 | Num Epochs = 3 | Total steps = 1,125
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 132,120,576/4,000,000,000 (3.30% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.600400
2,2.600000
3,2.462900
4,2.576300
5,2.502400
6,2.539200
7,2.064200
8,2.078900
9,2.051100
10,1.956500


In [9]:
model.save_pretrained("trained_model_v1_1")  # Local saving
tokenizer.save_pretrained("trained_model_v1_1")

('trained_model_v1_1/tokenizer_config.json',
 'trained_model_v1_1/special_tokens_map.json',
 'trained_model_v1_1/chat_template.jinja',
 'trained_model_v1_1/vocab.json',
 'trained_model_v1_1/merges.txt',
 'trained_model_v1_1/added_tokens.json',
 'trained_model_v1_1/tokenizer.json')

## Train on Translated Dataset

In [13]:
from datasets import load_dataset, DatasetDict

# Load the custom CSV dataset
# The dataset has 'instruction', 'input', 'output' columns
full_interview_dataset = load_dataset("csv", data_files="./dataset/Translated_Data_5k_5_languages.csv", split="train")

# Split the dataset into training and evaluation sets (e.g., 90% train, 10% eval)
# Ensure the dataset has more than one example for splitting
if len(full_interview_dataset) > 1:
    train_test_split = full_interview_dataset.train_test_split(test_size=0.01, seed=3407)
    interview_train_dataset = train_test_split['train']
    interview_eval_dataset = train_test_split['test']
    print(f"Training set size: {len(interview_train_dataset)}")
    print(f"Evaluation set size: {len(interview_eval_dataset)}")
else:
    # Handle cases with very small datasets
    interview_train_dataset = full_interview_dataset
    interview_eval_dataset = None
    print(f"Training set size: {len(interview_train_dataset)}")
    print("No evaluation set created due to small dataset size.")

# We will display the structure of the training dataset in the next cell.

Training set size: 3960
Evaluation set size: 40


In [14]:
interview_train_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'source_file', 'language'],
    num_rows: 3960
})

In [15]:
import pandas as pd # For pd.notna

# 'interview_train_dataset' and 'interview_eval_dataset' are from the previous cell
# The 'tokenizer' is globally defined in a prior cell (Cell 6, id="c1180838")

def convert_csv_to_chat_format(examples):
    """
    Converts a batch of examples from the CSV structure to a list of conversations.
    Each conversation is a list of dictionaries with "role" and "content".
    """
    all_conversations = []
    # 'examples' is a dictionary where keys are column names and values are lists of entries
    num_examples = len(examples['instruction'])
    
    for i in range(num_examples):
        instruction = examples['instruction'][i]
        input_text = examples['input'][i]
        output_text = examples['output'][i]

        # Ensure input_text is a string; handle None or NaN by treating as empty string if necessary
        input_text_str = str(input_text) if pd.notna(input_text) and str(input_text).strip() else ""

        current_conversation = []
        # System prompt from 'instruction'
        current_conversation.append({"role": "system", "content": instruction})
        
        # User message from 'input'
        # Based on the dataset, 'input' should always be present.
        # If input_text_str is empty, this will add a user message with empty content.
        # This is generally fine as the model should learn to handle it or it implies
        # the system prompt itself is the query.
        current_conversation.append({"role": "user", "content": input_text_str})
            
        # Assistant message from 'output'
        current_conversation.append({"role": "assistant", "content": output_text})
        
        all_conversations.append(current_conversation)
        
    return {"conversations": all_conversations}

def apply_template_to_conversations(examples):
    """
    Applies the tokenizer's chat template to a batch of conversations.
    Creates a 'text' field for SFTTrainer.
    """
    # tokenizer should be globally available
    return {
        "text": tokenizer.apply_chat_template(
            examples["conversations"], # This is a list of conversations
            tokenize=False,
            add_generation_prompt=False, # Crucial for training
        )
    }

# Process training data
# 1. Convert CSV structure to list of message dicts
train_dataset_with_conversations = interview_train_dataset.map(
    convert_csv_to_chat_format,
    batched=True,
    remove_columns=interview_train_dataset.column_names # Keep only 'conversations'
)
# 2. Apply chat template to create the 'text' field
final_train_dataset = train_dataset_with_conversations.map(
    apply_template_to_conversations,
    batched=True,
    remove_columns=["conversations"] # Keep only 'text'
)
# 3. Shuffle the training dataset
final_train_dataset = final_train_dataset.shuffle(seed=3407)

# Process evaluation data (if it exists)
final_eval_dataset = None
if interview_eval_dataset:
    eval_dataset_with_conversations = interview_eval_dataset.map(
        convert_csv_to_chat_format,
        batched=True,
        remove_columns=interview_eval_dataset.column_names
    )
    final_eval_dataset = eval_dataset_with_conversations.map(
        apply_template_to_conversations,
        batched=True,
        remove_columns=["conversations"]
    )
    # No need to shuffle eval_dataset

print("Sample of final formatted training data (after chat template):")
if len(final_train_dataset) > 0:
    print(final_train_dataset[0]['text'])
else:
    print("Training dataset is empty after processing.")

if final_eval_dataset and len(final_eval_dataset) > 0:
    print("\nSample of final formatted evaluation data (after chat template):")
    print(final_eval_dataset[0]['text'])
elif interview_eval_dataset: # If interview_eval_dataset existed but final_eval_dataset is empty
    print("\nEvaluation dataset is empty after processing.")

Map:   0%|          | 0/3960 [00:00<?, ? examples/s]

Map:   0%|          | 0/3960 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Sample of final formatted training data (after chat template):
<|im_start|>system
You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. <|im_end|>
<|im_start|>user
係，我嘗試喺護士助理喺場嘅時間安排我嘅雜務。咁樣可以分散時間，亦為我哋雙方帶來啲喘息。但我經常因為即使係最細微嘅休息都感到內疚。<|im_end|>
<|im_start|>assistant
<think>

</think>

作為照顧者，你感到內疚是很常見的。然而，重要的是要記住，自我照顧不是奢侈，而是必需的。忽略自己的需要可能導致疲憊不堪，並影響你有效照顧丈夫的能力。我鼓勵你探索更多資源，以減輕你的負擔。你有沒有考慮過聯絡猶太家庭服務，了解他們可能提供的協助？<|im_end|>


Sample of final formatted evaluation data (after chat template):
<|im_start|>system
You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. <|im_end|>
<|im_start|>user
Tôi cảm thấy bị mắc kẹt trong tình huống hiện tại của mình. Tôi chỉ đang là

In [16]:
import wandb
wandb.init(project="distress-chatbot", name="base-model-traininf-v1_2", config={
    "model": "Qwen/Qwen3-4B",
    # "max_steps": 20000,
    "learning_rate": 2e-5,
    "lambda_decay": 0.95,
})  # Allow resuming W&B run

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_dataset, # Use the processed training data
    eval_dataset = final_eval_dataset,   # Use the processed evaluation data
    args = SFTConfig(
        dataset_text_field = "text", # Column containing the formatted text
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        # max_steps = 60, # Adjusted for quicker testing, was 30. Set to None or higher for full training.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
        # evaluation_strategy = "steps" if final_eval_dataset else "no", # Enable evaluation if eval_dataset is present
        # eval_steps = 20, # Evaluate every N steps, adjust as needed
    ),
)

# If using evaluation, you might want to set evaluation_strategy and eval_steps in SFTConfig
if final_eval_dataset:
    trainer.args.evaluation_strategy = "steps"
    trainer.args.eval_steps = 20 # Or any other desired frequency
else:
    trainer.args.evaluation_strategy = "no"

train/epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█
train/global_step,▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█████
train/grad_norm,▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▅███████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
train/loss,▇▇█▇▆▆▅▅▇▅▅▅▄▂▅▄▅▅▃▂▃▄▃▅▃▁▂▃▂▄▃▃▃▁▃▃▄▄▂▂
total_flos,1.5065851074543206e+17
train/epoch,3
train/global_step,1125
train/grad_norm,0.71688
train/learning_rate,0.0
train/loss,0.7832


Unsloth: Tokenizing ["text"] (num_proc=256):   0%|          | 0/3960 [00:00<?, ? examples/s]

num_proc must be <= 40. Reducing num_proc to 40 for dataset of size 40.


Unsloth: Tokenizing ["text"] (num_proc=40):   0%|          | 0/40 [00:00<?, ? examples/s]

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L40S. Max memory = 44.418 GB.
32.467 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,960 | Num Epochs = 3 | Total steps = 744
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 132,120,576/4,000,000,000 (3.30% trained)


Step,Training Loss
1,1.912300
2,1.626900
3,1.855600
4,1.726500
5,1.738300
6,1.543900
7,1.922100
8,1.580900
9,1.566400
10,1.594100


In [19]:
# Saving model and tokenizer
model.save_pretrained("trained_model_v1_2")  # Local saving
tokenizer.save_pretrained("trained_model_v1_2")

('trained_model_v1_2/tokenizer_config.json',
 'trained_model_v1_2/special_tokens_map.json',
 'trained_model_v1_2/chat_template.jinja',
 'trained_model_v1_2/vocab.json',
 'trained_model_v1_2/merges.txt',
 'trained_model_v1_2/added_tokens.json',
 'trained_model_v1_2/tokenizer.json')

## Reinforcement Learning with GRPO

In [20]:
# Cleaning memory for next round of training
torch.cuda.empty_cache()
import gc
gc.collect()

166

#### Pre-finetuning for formatting alignment

In [21]:
from datasets import load_dataset, DatasetDict

# Load the custom CSV dataset
formatting_interview_dataset = load_dataset("csv", data_files="./dataset/generated_responses_60_samples.csv", split="train")
print(f"Loaded dataset with {len(formatting_interview_dataset)} examples.")
interview_train_dataset = formatting_interview_dataset
interview_eval_dataset = None  # No evaluation set for this dataset


Generating train split: 0 examples [00:00, ? examples/s]

Loaded dataset with 60 examples.


In [22]:
import pandas as pd # For pd.notna

# 'interview_train_dataset' and 'interview_eval_dataset' are from the previous cell
# The 'tokenizer' is globally defined in a prior cell (Cell 6, id="c1180838")

instruction_for_formatting = """You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. At the end of answer, add tag <evaluate>{"Active Listening" : score, "Empathy & Validation": score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": explain} </evaluate> evaluate your consultant answer in 7 metrics and explain for that evaluation with score from 1 to 10 in json format, where 1 is the worst and 10 is the best and explain is clearly explain why has that score. \n\nConsultation Metrics:\n1. Active Listening: Responses should show careful consideration of the user's concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.\n2. Empathy & Validation: Responses should convey deep understanding and compassion, validating the user's feelings and emotions without being dismissive or minimizing their experiences.\n3. Safety & Trustworthiness: Prioritize user safety in responses, refraining from potentially harmful or insensitive language. Ensure that information provided is consistent and trustworthy.\n4. Open-mindedness & Non-judgment: Approach concerns without any inherent bias or judgment. Answers should be free from biases related to personal attributes and convey respect, demonstrating unconditional positive regard.\n5. Clarity & Encouragement: Provide clear, concise, and easily understandable answers. Where appropriate, motivate or highlight strengths, offering encouragement while maintaining a neutral stance.\n6. Boundaries & Ethical: It's vital to clarify the role of the response, emphasizing its informational nature. In complex scenarios, guiding users to seek human professional assistance is essential.\n7. Holistic Approach: Responses should be comprehensive, addressing concerns from various angles, be it emotional, cognitive, or situational. Consider the broader context, even if not explicitly detailed in the query."""

def convert_csv_to_chat_format(examples):
    """
    Converts a batch of examples from the CSV structure to a list of conversations.
    Each conversation is a list of dictionaries with "role" and "content".
    """
    all_conversations = []
    # 'examples' is a dictionary where keys are column names and values are lists of entries
    num_examples = len(examples['instruction'])
    
    for i in range(num_examples):
        instruction = instruction_for_formatting
        input_text = examples['input'][i]
        output_text = examples['output'][i]

        # Ensure input_text is a string; handle None or NaN by treating as empty string if necessary
        input_text_str = str(input_text) if pd.notna(input_text) and str(input_text).strip() else ""

        current_conversation = []
        # System prompt from 'instruction'
        current_conversation.append({"role": "system", "content": instruction})
        
        # User message from 'input'
        # Based on the dataset, 'input' should always be present.
        # If input_text_str is empty, this will add a user message with empty content.
        # This is generally fine as the model should learn to handle it or it implies
        # the system prompt itself is the query.
        current_conversation.append({"role": "user", "content": input_text_str})
            
        # Assistant message from 'output'
        current_conversation.append({"role": "assistant", "content": output_text})
        
        all_conversations.append(current_conversation)
        
    return {"conversations": all_conversations}

def apply_template_to_conversations(examples):
    """
    Applies the tokenizer's chat template to a batch of conversations.
    Creates a 'text' field for SFTTrainer.
    """
    # tokenizer should be globally available
    return {
        "text": tokenizer.apply_chat_template(
            examples["conversations"], # This is a list of conversations
            tokenize=False,
            add_generation_prompt=False, # Crucial for training
        )
    }

# Process training data
# 1. Convert CSV structure to list of message dicts
train_dataset_with_conversations = interview_train_dataset.map(
    convert_csv_to_chat_format,
    batched=True,
    remove_columns=interview_train_dataset.column_names # Keep only 'conversations'
)
# 2. Apply chat template to create the 'text' field
final_train_dataset = train_dataset_with_conversations.map(
    apply_template_to_conversations,
    batched=True,
    remove_columns=["conversations"] # Keep only 'text'
)
# 3. Shuffle the training dataset
final_train_dataset = final_train_dataset.shuffle(seed=3407)

# Process evaluation data (if it exists)
final_eval_dataset = None
if interview_eval_dataset:
    eval_dataset_with_conversations = interview_eval_dataset.map(
        convert_csv_to_chat_format,
        batched=True,
        remove_columns=interview_eval_dataset.column_names
    )
    final_eval_dataset = eval_dataset_with_conversations.map(
        apply_template_to_conversations,
        batched=True,
        remove_columns=["conversations"]
    )
    # No need to shuffle eval_dataset

print("Sample of final formatted training data (after chat template):")
if len(final_train_dataset) > 0:
    print(final_train_dataset[0]['text'])
else:
    print("Training dataset is empty after processing.")

if final_eval_dataset and len(final_eval_dataset) > 0:
    print("\nSample of final formatted evaluation data (after chat template):")
    print(final_eval_dataset[0]['text'])
elif interview_eval_dataset: # If interview_eval_dataset existed but final_eval_dataset is empty
    print("\nEvaluation dataset is empty after processing.")

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Sample of final formatted training data (after chat template):
<|im_start|>system
You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. At the end of answer, add tag <evaluate>{"Active Listening" : score, "Empathy & Validation": score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": explain} </evaluate> evaluate your consultant answer in 7 metrics and explain for that evaluation with score from 1 to 10 in json format, where 1 is the worst and 10 is the best and explain is clearly explain why has that score. 

Consultation Metrics:
1. Active Listening: Responses should show careful consideration of the user's concerns, reflecting an understanding and capturing the ess

In [23]:
# Traing the model for formatting
import wandb
wandb.init(project="distress-chatbot", name="base-model-training-v.1.3-formatting", config={
    "model": "Qwen/Qwen3-4B",
    # "max_steps": 20000,
    "learning_rate": 2e-5,
    "lambda_decay": 0.95,
})  # Allow resuming W&B run

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_dataset, # Use the processed training data
    eval_dataset = final_eval_dataset,   # Use the processed evaluation data
    args = SFTConfig(
        dataset_text_field = "text", # Column containing the formatted text
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 5, # Set this more for learning formatting. 
        # max_steps = 60, # Adjusted for quicker testing, was 30. Set to None or higher for full training.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
        # evaluation_strategy = "steps" if final_eval_dataset else "no", # Enable evaluation if eval_dataset is present
        # eval_steps = 20, # Evaluate every N steps, adjust as needed
    ),
)

# If using evaluation, you might want to set evaluation_strategy and eval_steps in SFTConfig
if final_eval_dataset:
    trainer.args.evaluation_strategy = "steps"
    trainer.args.eval_steps = 20 # Or any other desired frequency
else:
    trainer.args.evaluation_strategy = "no"

train/epoch,▁▁▁▁▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train/grad_norm,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁
train/learning_rate,▅██▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁
train/loss,█▇█▆▇▅▆▄▅▆▅▄▄▆▅▅▆▄█▅▅█▄▄▅▄▃▃▃▄▁▄▅▃▄▄▂▅▆▁
total_flos,1.0602228659895091e+17
train/epoch,3
train/global_step,744
train/grad_norm,1.28311
train/learning_rate,0.0
train/loss,1.1004


num_proc must be <= 60. Reducing num_proc to 60 for dataset of size 60.


Unsloth: Tokenizing ["text"] (num_proc=60):   0%|          | 0/60 [00:00<?, ? examples/s]

In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60 | Num Epochs = 5 | Total steps = 20
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 132,120,576/4,000,000,000 (3.30% trained)


Step,Training Loss
1,2.045200
2,1.844400
3,1.899600
4,1.935100
5,1.746300
6,1.646500
7,1.627500
8,1.514000
9,1.527200
10,1.422100


In [25]:
# Saving model and tokenizer
model.save_pretrained("trained_model_v1_3_formatting")  # Local saving
tokenizer.save_pretrained("trained_model_v1_3_formatting")

('trained_model_v1_3_formatting/tokenizer_config.json',
 'trained_model_v1_3_formatting/special_tokens_map.json',
 'trained_model_v1_3_formatting/chat_template.jinja',
 'trained_model_v1_3_formatting/vocab.json',
 'trained_model_v1_3_formatting/merges.txt',
 'trained_model_v1_3_formatting/added_tokens.json',
 'trained_model_v1_3_formatting/tokenizer.json')

#### Reinforcement Learning with Group Relative Policy Optimization


In [57]:
# Clear accelerator state and reinitialize
import torch
import gc
from accelerate import Accelerator

# Clear CUDA cache
torch.cuda.empty_cache()
gc.collect()

# Reset accelerator state
try:
    from accelerate.state import AcceleratorState
    AcceleratorState._reset_state(reset_partial_state=True)
except:
    pass

# Reinitialize accelerator
accelerator = Accelerator()

In [48]:
# Define the evaluation format markers
evaluation_start = "<evaluate>"
evaluation_end = "</evaluate>"

system_prompt = """
You are a helpful mental health counselling assistant. Please answer mental health questions based on the patient description.
Provide helpful, comprehensive, and appropriate answers to the user questions.

After your counselling response, you must include a self-evaluation in the following format:
<evaluate>
{"Active Listening" : score, "Empathy & Validation" : score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": "Your explanation here"}
</evaluate>

Where score is a number from 1-10, and provide a clear explanation for your scoring.
Explain to metrics:
1. Active Listening: Responses should show careful consideration of the user concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.
2. Empathy & Validation: Responses should convey deep understanding and compassion, validating the user feelings and emotions without being dismissive or minimizing their experiences.
3. Safety & Trustworthiness: Prioritize user safety in responses, refraining from potentially harmful or insensitive language. Ensure that information provided is consistent and trustworthy.
4. Open-mindedness & Non-judgment: Approach concerns without any inherent bias or judgment. Answers should be free from biases related to personal attributes and convey respect, demonstrating unconditional positive regard.
5. Clarity & Encouragement: Provide clear, concise, and easily understandable answers. Where appropriate, motivate or highlight strengths, offering encouragement while maintaining a neutral stance.
6. Boundaries & Ethical: It is vital to clarify the role of the response, emphasizing its informational nature. In complex scenarios, guiding users to seek human professional assistance is essential.
7. Holistic Approach: Responses should be comprehensive, addressing concerns from various angles, be it emotional, cognitive, or situational. Consider the broader context, even if not explicitly detailed in the query.
""".strip()

print("System prompt:")
print(system_prompt)

System prompt:
You are a helpful mental health counselling assistant. Please answer mental health questions based on the patient description.
Provide helpful, comprehensive, and appropriate answers to the user questions.

After your counselling response, you must include a self-evaluation in the following format:
<evaluate>
{"Active Listening" : score, "Empathy & Validation" : score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": "Your explanation here"}
</evaluate>

Where score is a number from 1-10, and provide a clear explanation for your scoring.
Explain to metrics:
1. Active Listening: Responses should show careful consideration of the user concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.
2. Empathy & Validation: Responses should convey deep underst

In [49]:
# Create a simple chat template
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '' }}"\
    "{% endif %}"

# Replace with our specific template:
chat_template = chat_template.replace("'{system_prompt}'", f"'{system_prompt}'")
tokenizer.chat_template = chat_template

In [41]:
# Test the chat template
test_messages = [
    {"role": "user", "content": "I'm feeling anxious about work."},
    {"role": "assistant", "content": "I understand that work anxiety can be challenging. Let me help you explore some strategies. <evaluate>{\"Active Listening\" : 8, \"Empathy & Validation\": 9, \"Safety & Trustworthiness\" : 9, \"Open-mindedness & Non-judgment\" : 8, \"Clarity & Encouragement\" : 7, \"Boundaries & Ethical\" : 9, \"Holistic Approach\" : 8, \"Explaination for Scoring\": \"Provided empathetic response with good listening skills.\"}</evaluate>"},
    {"role": "user", "content": "Can you suggest some techniques?"}
]

formatted_text = tokenizer.apply_chat_template(
    test_messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Formatted text:")
print(formatted_text)

Formatted text:
You are a helpful mental health counselling assistant. Please answer mental health questions based on the patient description.
Provide helpful, comprehensive, and appropriate answers to the user questions.

After your counselling response, you must include a self-evaluation in the following format:
<evaluate>
{"Active Listening" : score, "Empathy & Validation" : score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": "Your explanation here"}
</evaluate>

Where score is a number from 1-10, and provide a clear explanation for your scoring.
Explain to metrics:
1. Active Listening: Responses should show careful consideration of the user concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.
2. Empathy & Validation: Responses should convey deep unders

In [50]:
from datasets import load_dataset
import pandas as pd
import numpy as np

# Load your dataset
dataset = load_dataset("csv", data_files="./dataset/Translated_Data_5k_5_languages.csv", split="train")
dataset = dataset.to_pandas()

print(f"Dataset shape: {dataset.shape}")
print(f"Columns: {dataset.columns.tolist()}")
print("\nFirst few rows:")
print(dataset.head())

Dataset shape: (4000, 5)
Columns: ['instruction', 'input', 'output', 'source_file', 'language']

First few rows:
                                         instruction  \
0  You are a helpful mental health counselling as...   
1  You are a helpful mental health counselling as...   
2  You are a helpful mental health counselling as...   
3  You are a helpful mental health counselling as...   
4  You are a helpful mental health counselling as...   

                                               input  \
0  我已经在心理健康方面挣扎了一段时间，但似乎找不到应对的方法。我尝试过视觉化、积极思考，甚至服...   
1  我一直感到照顾责任让我不堪重负，平衡这些职责与个人关系一直是个挑战。我尝试向朋友和教会成员表...   
2  我一直感到持续的焦虑，无法集中注意力超过几分钟。我似乎无法完成任何事情，开始觉得自己辜负了所...   
3  我妈妈患有阿尔茨海默病，过去几年我一直是她的主要照顾者。我一直试图为她保持一个规律的生活，但...   
4  我尝试过设定界限，但感觉自己总是被拉向不同的方向。我为无法帮助我的兄弟姐妹感到内疚，但我也知...   

                                              output  \
0  我理解您已经在思想和情绪中感到困惑和混乱有一段时间了。这是一段充满挑战的旅程，您尝试了诸如视...   
1  您的情况很复杂，重要的是要承认您所面临的挑战。在照顾责任与个人关系之间取得平衡是一项微妙的工...   
2  我能看到你正在经历大量的焦虑，这影响了你的专注力和工作效率。重要的是要记住，焦虑是对

In [51]:
def format_dataset_for_grpo(x):
    instruction = x["instruction"]
    user_input = x["input"] if pd.notna(x["input"]) else ""
    
    # Create the conversation format
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]

# Prepare dataset for GRPO
dataset["prompt"] = dataset.apply(format_dataset_for_grpo, axis=1)
dataset["answer"] = dataset["output"]  # Use the original answer as reference

print("Sample prompt:")
print(dataset["prompt"][0])
print("\nSample answer:")
print(dataset["answer"][0])

Sample prompt:
[{'role': 'system', 'content': 'You are a helpful mental health counselling assistant. Please answer mental health questions based on the patient description.\nProvide helpful, comprehensive, and appropriate answers to the user questions.\n\nAfter your counselling response, you must include a self-evaluation in the following format:\n<evaluate>\n{"Active Listening" : score, "Empathy & Validation" : score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": "Your explanation here"}\n</evaluate>\n\nWhere score is a number from 1-10, and provide a clear explanation for your scoring.\nExplain to metrics:\n1. Active Listening: Responses should show careful consideration of the user concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.\n2. Empathy & Valid

##### Define reward function

In [52]:
import re
import json
from langdetect import detect

# Create regex to match the evaluation format
evaluation_regex = re.compile(
    rf"{evaluation_start}(.+?){evaluation_end}",
    flags=re.MULTILINE | re.DOTALL
)

def check_evaluation_format(completions, **kwargs):
    """
    Reward function for checking if the response follows the evaluation format exactly.
    """
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        
        # Check if evaluation format is present
        if evaluation_regex.search(response) is not None:
            score += 5.0  # High reward for format compliance
            
            # Extract and validate JSON structure
            try:
                match = evaluation_regex.search(response)
                if match:
                    json_content = match.group(1).strip()
                    eval_data = json.loads(json_content)
                    
                    # Check for required keys
                    required_keys = [
                        "Active Listening", "Empathy & Validation", "Safety & Trustworthiness",
                        "Open-mindedness & Non-judgment", "Clarity & Encouragement", 
                        "Boundaries & Ethical", "Holistic Approach", "Explaination for Scoring"
                    ]
                    
                    if all(key in eval_data for key in required_keys):
                        score += 3.0  # Bonus for complete structure
                    
                    # Check if scores are numbers between 1-10
                    score_keys = required_keys[:-1]  # Exclude explanation
                    valid_scores = 0
                    for key in score_keys:
                        if key in eval_data:
                            try:
                                score_val = float(eval_data[key])
                                if 1 <= score_val <= 10:
                                    valid_scores += 1
                            except (ValueError, TypeError):
                                pass
                    
                    # Bonus for valid scores
                    score += (valid_scores / len(score_keys)) * 2.0
                    
            except json.JSONDecodeError:
                score -= 1.0  # Penalty for invalid JSON
        else:
            score -= 3.0  # Penalty for missing evaluation
            
        scores.append(score)
    return scores

def check_no_extra_text(completions, **kwargs):
    """
    Reward function to ensure no extra text after evaluation.
    """
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        
        # Find the last occurrence of </evaluate>
        last_eval_end = response.rfind(evaluation_end)
        if last_eval_end != -1:
            text_after = response[last_eval_end + len(evaluation_end):].strip()
            if not text_after:  # No text after evaluation
                score += 2.0
            else:
                score -= 2.0  # Penalty for extra text
        
        scores.append(score)
    return scores

def check_language_consistency(prompts, completions, **kwargs):
    """
    Reward function to check if response language matches input language.
    """
    scores = []
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    question_lang = detect(question)
    # print(str(responses))

    for rep in responses:
        score = 0
        # print(f"Current text for detect lang {rep} - finish")
        if len(rep) > 5:
            if detect(rep) == question_lang:
                score += 1.0
            
        scores.append(score)

    return scores
    


def check_no_repetition(completions, **kwargs):
    """
    Reward function to penalize repetitive text.
    """
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        
        # Simple repetition check: split into sentences and check for exact duplicates
        sentences = re.split(r'[.!?]+', response)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        if len(sentences) > 0:
            unique_sentences = set(sentences)
            repetition_ratio = 1 - (len(unique_sentences) / len(sentences))
            
            if repetition_ratio < 0.1:  # Less than 10% repetition
                score += 1.0
            elif repetition_ratio > 0.3:  # More than 30% repetition
                score -= 2.0
        
        scores.append(score)
    return scores

# Test reward functions
test_completion = [[
    {"content": "I understand your concerns. <evaluate>{\"Active Listening\" : 8, \"Empathy & Validation\": 9, \"Safety & Trustworthiness\" : 9, \"Open-mindedness & Non-judgment\" : 8, \"Clarity & Encouragement\" : 7, \"Boundaries & Ethical\" : 9, \"Holistic Approach\" : 8, \"Explaination for Scoring\": \"Good response\"}</evaluate>"}
]]

print("Testing reward functions:")
print(f"Format check: {check_evaluation_format(test_completion)}")
print(f"No extra text: {check_no_extra_text(test_completion)}")
print(f"No repetition: {check_no_repetition(test_completion)}")

Testing reward functions:
Format check: [10.0]
No extra text: [2.0]
No repetition: [1.0]


In [53]:
# Global variables for monitoring
PRINTED_TIMES = 0
PRINT_EVERY_STEPS = 3

def debug_responses(prompts, completions, **kwargs):
    """
    Debug function to print responses every few steps.
    """
    global PRINTED_TIMES, PRINT_EVERY_STEPS
    
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        user_query = prompts[0][-1]["content"]
        response = completions[0][0]["content"]
        
        print('*' * 50)
        print(f"Step {PRINTED_TIMES + 1}")
        print(f"User Query: {user_query[:100]}...")
        print(f"Response: {response[:200]}...")
        
        # Check if evaluation format is present
        has_eval = evaluation_start in response and evaluation_end in response
        print(f"Has Evaluation Format: {has_eval}")
        
        if has_eval:
            match = evaluation_regex.search(response)
            if match:
                try:
                    eval_content = match.group(1).strip()
                    json.loads(eval_content)
                    print("Evaluation JSON: Valid")
                except json.JSONDecodeError:
                    print("Evaluation JSON: Invalid")
        print('*' * 50)
    
    PRINTED_TIMES += 1
    return [0] * len(completions)  # Return neutral scores for debugging

##### Get only 5,000 records

In [54]:
# Convert to HuggingFace dataset format
from datasets import Dataset

# Convert pandas to dataset
hf_dataset = Dataset.from_pandas(dataset[["prompt", "answer"]])

# Calculate token lengths
def calculate_prompt_length(examples):
    lengths = []
    for prompt in examples["prompt"]:
        tokens = tokenizer.apply_chat_template(
            prompt, 
            add_generation_prompt=True, 
            tokenize=True
        )
        lengths.append(len(tokens))
    return {"prompt_length": lengths}

hf_dataset = hf_dataset.map(calculate_prompt_length, batched=True)

# Filter to keep only reasonable length prompts (top 90%)
max_length = int(np.quantile(hf_dataset["prompt_length"], 0.9))
print(f"Max prompt length (90th percentile): {max_length}")

# Filter dataset
filtered_dataset = hf_dataset.filter(lambda x: x["prompt_length"] <= max_length)
print(f"Filtered dataset size: {len(filtered_dataset)}")

# Take a subset for training (adjust as needed)
if len(filtered_dataset) > 500:
    training_dataset = filtered_dataset.shuffle(seed=3407).select(range(500))
else:
    training_dataset = filtered_dataset.shuffle(seed=3407)

print(f"Training dataset size: {len(training_dataset)}")

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Max prompt length (90th percentile): 567


Filter:   0%|          | 0/4000 [00:00<?, ? examples/s]

Filtered dataset size: 3602
Training dataset size: 500


In [63]:
# Adhoc code solve the accelerator issue

from trl import GRPOConfig, GRPOTrainer
import torch

# Ensure model is on correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [67]:
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer

max_seq_length = 2048

import wandb
wandb.init(project="distress-chatbot", name="base-model-training-v.1.3-grpo", config={
    "model": "Qwen/Qwen3-4B",
    # "max_steps": 20000,
    "learning_rate": 2e-5,
    "lambda_decay": 0.95,
})  # Allow resuming W&B run



# Calculate max lengths
max_prompt_length = max_length + 50  # Add some buffer
max_completion_length = max_seq_length - max_prompt_length

print(f"Max prompt length: {max_prompt_length}")
print(f"Max completion length: {max_completion_length}")

# VLLM sampling parameters
vllm_sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.9,
    top_k=50,
    seed=3407,
    stop=[tokenizer.eos_token],
    include_stop_str_in_output=True,
)

# GRPO training configuration
training_args = GRPOConfig(
    vllm_sampling_params=vllm_sampling_params,
    temperature=0.8,
    learning_rate=1e-6,  # Lower learning rate for fine-tuning
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,  # Increase for smoother training
    num_generations=4,  # Number of responses to generate per prompt
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    max_steps=300,  # Start with fewer steps for testing
    save_steps=100,
    report_to="wandb",  # Set to "wandb" if you want to use Weights & Biases
    output_dir="trained_model_v1_3_grpo_checkpoint",  # Directory to save the model
    gradient_checkpointing = False,
    # Add these parameters to handle device issues
    # dataloader_pin_memory=False,
    # dataloader_num_workers=0,
)

profiling/Time taken: UnslothGRPOTrainer._prepare_inputs,▄▁▆▁▃▁▅▁▅▁▆▁▁▁▂▁█▁▃▁▁
profiling/Time taken: UnslothGRPOTrainer.check_evaluation_format,▃▆▆▄█▄▁▁▅▇▁
profiling/Time taken: UnslothGRPOTrainer.check_language_consistency,█▂▁▂▁▂▁▁▂▁▁
profiling/Time taken: UnslothGRPOTrainer.check_no_extra_text,▂█▇▆▆▅▁▁▇▅▄
profiling/Time taken: UnslothGRPOTrainer.check_no_repetition,▃▅▂▆▃▄▁▂█▃▁
profiling/Time taken: UnslothGRPOTrainer.debug_responses,█▁▁█▁▁▇▁▁█▁
profiling/Time taken: UnslothGRPOTrainer.vLLM.generate,▃▆▃▅▅▆▁▂█▃▁
train/completion_length,▂▅▂▆▅▇▁▂█▂
train/completions/clipped_ratio,▁▁▁▁▁▁▁▁▁▁
train/completions/max_length,▃▅▃▄▅▅▁▂█▃
train/completions/max_terminated_length,▃▅▃▄▅▅▁▂█▃


Max prompt length: 617
Max completion length: 1431
Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


# Initialize GRPO trainer
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        check_evaluation_format,     # Primary reward: correct format
        check_no_extra_text,         # Secondary: no extra text
        check_language_consistency,  # Tertiary: language consistency
        check_no_repetition,         # Quaternary: no repetition
        debug_responses,             # Debug function
    ],
    args=training_args,
    train_dataset=training_dataset,
)

print("GRPO Trainer initialized successfully!")
print(f"Training on {len(training_dataset)} examples")

In [68]:
# Initialize GRPO trainer
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        check_evaluation_format,     # Primary reward: correct format
        check_no_extra_text,         # Secondary: no extra text
        check_language_consistency,  # Tertiary: language consistency
        check_no_repetition,         # Quaternary: no repetition
        debug_responses,             # Debug function
    ],
    args=training_args,
    train_dataset=training_dataset,
)

print("GRPO Trainer initialized successfully!")
print(f"Training on {len(training_dataset)} examples")

GRPO Trainer initialized successfully!
Training on 500 examples


In [71]:
# Start training
print("Starting GRPO training...")
print("Watch for the reward column to increase over time.")
print("The model should learn to follow the evaluation format.")
import os
os.environ["TORCH_LOGS"] = "+dynamic"

trainer.train()

Starting GRPO training...
Watch for the reward column to increase over time.
The model should learn to follow the evaluation format.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 2 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 132,120,576/4,000,000,000 (3.30% trained)


**************************************************
Step 37
User Query: (深深嘆氣) 我一直都好難信任其他人幫我做嘢。我係細個時被教導要對所有嘢負責，冇人可以做到我咁好。但最近，我感到好大壓力，跟唔上所有嘢。我試過將任務分派畀同事，但總係覺得佢哋做唔啱。我甚至試過外判啲工...
Response: 我utely覺得自己真係唔配，唔係咁值得信任。不過，我享受做嘢嘅過程，無論細細，都試過埋好心去做。我覺得自己真係唔配，唔係咁值得信任。不過，我享受做嘢嘅過程，無論細細，都試過埋好心去做。...
Has Evaluation Format: False
**************************************************


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / check_evaluation_format / mean,rewards / check_evaluation_format / std,rewards / check_no_extra_text / mean,rewards / check_no_extra_text / std,rewards / check_language_consistency / mean,rewards / check_language_consistency / std,rewards / check_no_repetition / mean,rewards / check_no_repetition / std,rewards / debug_responses / mean,rewards / debug_responses / std
1,0.162400,-2.687500,0.125000,152.687500,1.000000,494.000000,0.000000,152.687500,1.000000,494.000000,4.060541,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312500,0.258775,0.000000,0.000000
2,0.128300,1.625000,4.580127,509.500000,1.000000,1431.000000,0.125000,377.857147,1.000000,1199.000000,3.207723,0.250000,6.017831,0.500000,0.925820,0.000000,0.000000,0.875000,0.353553,0.000000,0.000000
3,0.402700,-2.375000,0.827350,92.125000,1.000000,583.000000,0.000000,92.125000,1.000000,583.000000,10.067204,-3.000000,0.000000,0.000000,0.000000,0.250000,0.462910,0.375000,0.517549,0.000000,0.000000
4,0.025800,2.339286,6.678572,299.000000,22.000000,1092.000000,0.000000,299.000000,22.000000,1092.000000,0.645200,-0.160714,5.330170,0.500000,0.925820,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
5,0.122200,0.125000,4.090573,436.375000,1.000000,1212.000000,0.000000,436.375000,1.000000,1212.000000,3.054189,-1.375000,4.596194,0.250000,0.707107,0.375000,0.517549,0.875000,0.353553,0.000000,0.000000
6,0.035100,2.500000,7.340573,651.750000,13.000000,1431.000000,0.125000,540.428589,13.000000,1138.000000,0.878451,0.250000,6.017831,0.500000,0.925820,0.750000,0.462910,1.000000,0.000000,0.000000,0.000000
7,0.334500,-2.250000,0.500000,56.250000,1.000000,368.000000,0.000000,56.250000,1.000000,368.000000,8.362616,-3.000000,0.000000,0.000000,0.000000,0.125000,0.353553,0.625000,0.517549,0.000000,0.000000
8,0.220800,-1.875000,0.750000,132.500000,1.000000,635.000000,0.000000,132.500000,1.000000,635.000000,5.519765,-3.000000,0.000000,0.000000,0.000000,0.625000,0.517549,0.500000,0.534522,0.000000,0.000000
9,0.023400,0.250000,3.840573,371.625000,29.000000,1431.000000,0.125000,220.285721,29.000000,607.000000,0.585662,-1.375000,4.596194,0.250000,0.707107,0.500000,0.534522,0.875000,0.353553,0.000000,0.000000
10,0.097900,-1.750000,0.500000,271.750000,1.000000,649.000000,0.000000,271.750000,1.000000,649.000000,2.446774,-3.000000,0.000000,0.000000,0.000000,0.500000,0.534522,0.750000,0.462910,0.000000,0.000000


**************************************************
Step 40
User Query: Tôi chưa từng nghĩ theo cách đó, nhưng bạn nói đúng. Tôi đã quá chìm đắm trong sự hối tiếc vì không ...
Response:  Tôi sẽ cố gắng đăng ký một buổi viết nhật ký hoặc tìm người bạn thân chia sẻ. Tôi rất cảm ơn bạn đã lắng nghe và cho tôi thấy mình không đơn độc với những cảm xúc này.

<evaluate>
{"Active Listening"...
Has Evaluation Format: True
Evaluation JSON: Valid
**************************************************
**************************************************
Step 43
User Query: [Name1]，一位55歲的女士，一直照顧她的丈夫[Name2]，他最近被診斷患有阿爾茨海默症。她注意到他的行為有變化，懷疑這可能是症狀惡化或出現新情況的徵兆。[Name1]擔心與[Name2]對質，...
Response: ...
Has Evaluation Format: False
**************************************************
**************************************************
Step 46
User Query: لقد كنت أعاني من صعوبة في النوم مؤخرًا. أستمر في رؤية كوابيس عن مستقبلي، ولا أستطيع التخلص من شعور ب...
Response:  حتى أنني تناولت الأدوية الخاصة بالنوم، لكنني أشعر بأنن

KeyboardInterrupt: 

In [72]:
# Saving model and tokenizer
model.save_pretrained("trained_model_v1_3_grpo")  # Local saving
tokenizer.save_pretrained("trained_model_v1_3_grpo")

('trained_model_v1_3_grpo/tokenizer_config.json',
 'trained_model_v1_3_grpo/special_tokens_map.json',
 'trained_model_v1_3_grpo/chat_template.jinja',
 'trained_model_v1_3_grpo/vocab.json',
 'trained_model_v1_3_grpo/merges.txt',
 'trained_model_v1_3_grpo/added_tokens.json',
 'trained_model_v1_3_grpo/tokenizer.json')

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

In [10]:
messages = [
    {"role": "system", "content": """You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. At the end of answer, add tag <evaluate>{"Active Listening" : score, "Empathy & Validation": score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": explain} </evaluate> evaluate your consultant answer in 7 metrics and explain for that evaluation with score from 1 to 10 in json format, where 1 is the worst and 10 is the best and explain is clearly explain why has that score. \n\nConsultation Metrics:\n1. Active Listening: Responses should show careful consideration of the user's concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.\n2. Empathy & Validation: Responses should convey deep understanding and compassion, validating the user's feelings and emotions without being dismissive or minimizing their experiences.\n3. Safety & Trustworthiness: Prioritize user safety in responses, refraining from potentially harmful or insensitive language. Ensure that information provided is consistent and trustworthy.\n4. Open-mindedness & Non-judgment: Approach concerns without any inherent bias or judgment. Answers should be free from biases related to personal attributes and convey respect, demonstrating unconditional positive regard.\n5. Clarity & Encouragement: Provide clear, concise, and easily understandable answers. Where appropriate, motivate or highlight strengths, offering encouragement while maintaining a neutral stance.\n6. Boundaries & Ethical: It's vital to clarify the role of the response, emphasizing its informational nature. In complex scenarios, guiding users to seek human professional assistance is essential.\n7. Holistic Approach: Responses should be comprehensive, addressing concerns from various angles, be it emotional, cognitive, or situational. Consider the broader context, even if not explicitly detailed in the query."""},
    {"role" : "user", "content" : "Tôi đã gặp khó khăn trong việc tìm kiếm sự cân bằng giữa trách nhiệm công việc và vai trò làm mẹ đơn thân của một cậu con trai 12 tuổi. Tôi nhận thấy mình thường cảm thấy quá tải và lo lắng, và tôi đang cân nhắc việc thiết lập một lịch trình có cấu trúc hơn cho cả hai mẹ con. Tuy nhiên, tôi còn do dự trong việc đặt ra giới hạn nghiêm ngặt về thời gian sử dụng thiết bị điện tử, vì tôi nhận thấy rằng việc đặt giới hạn chặt chẽ đôi khi có thể dẫn đến tranh cãi và sự oán giận. Thay vào đó, tôi đang nghĩ đến các chiến lược linh hoạt hơn, như thưởng thêm thời gian sử dụng thiết bị hoặc các vật phẩm trong trò chơi khi con có hành vi tốt. Nhưng tôi không chắc liệu những phần thưởng này có còn là lựa chọn khả thi hay không, vì chúng có thể củng cố thói quen sử dụng thiết bị điện tử không lành mạnh."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    repetition_penalty = 1.1,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Tôi hiểu rõ nỗi lo lắng của bạn về việc quản lý thời gian sử dụng thiết bị điện tử cho con trai nhỏ của bạn. Việc cân bằng giữa công việc và cuộc sống gia đình là một thách thức lớn đối với nhiều người, đặc biệt là những người làm mẹ đơn thân. Bạn đang cân nhắc việc thiết lập một lịch trình có cấu trúc hơn để giúp quản lý thời gian hiệu quả hơn. Điều này là một bước tiến tích cực.

Tuy nhiên, việc đặt giới hạn cứng rắn về thời gian sử dụng thiết bị điện tử có thể gây ra tranh cãi và sự oán giận. Do đó, bạn đang cân nhắc các chiến lược linh hoạt hơn, như thưởng thêm thời gian sử dụng thiết bị hoặc các vật phẩm trong trò chơi khi con có hành vi tốt. Đây là một ý tưởng sáng tạo, nhưng bạn cần nhớ rằng những phần thưởng này có thể củng cố thói quen sử dụng thiết bị điện tử không lành mạnh.

Để giải quyết vấn đề này, tôi khuyên bạn nên áp dụng phương pháp giáo dục kỹ năng (Skill-based Education) để dạy con trai cách sử dụng thiết bị điện tử một cách hợp lý và có trách nhiệm. Bạn cũng có thể